In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.0 MB/s eta 0:00:00


In [ ]:
import transformers
import torch

In [ ]:
# Load a pre-trained transformer model and tokenizer
model = transformers.BertModel.from_pretrained('bert-base-uncased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
def sentiment(text):
  #Tokenize input text
  input_ids=tokenizer.encode(text,return_tensors='pt')
  

  #generate output from the model
  output =model(input_ids)

  #extract the hidden state of the last layer of the model
  last_hidden_state=output[0]

  #Use linear layer to map the hidden state to a single value
  linear_layer=torch.nn.Linear(last_hidden_state.shape[-1],1)
  sentiment=torch.sigmoid(linear_layer(last_hidden_state.squeeze())).max()
  print(sentiment)
  if sentiment>0.6:
    print('Positive sentiment')
  elif sentiment<0.4:
    print('Negative sentiment')
  else:
    print("neutral sentiment")

In [ ]:
sentiment('I hate you very much')

tensor(0.5595, grad_fn=<MaxBackward1>)
neutral sentiment


In [ ]:
import numpy as np

from scipy.stats import mode

def emotions(text):
  #Define the emotions to classify
  emotions=['happy','sad','angry','neutral']

  #tokenizer input text
  input_ids=tokenizer.encode(text,return_tensors='pt')

  #Genertae output from the model
  output=model(input_ids)

  #extract the hidden state of the laast layer of the model
  last_hidden_state=output[0]
  
  #use a linear layer to map the hidden state to as probability  
  linear_layer=torch.nn.Linear(last_hidden_state.shape[-1],len(emotions))
  probabilities = torch.softmax(linear_layer(last_hidden_state.squeeze()).view(-1,len(emotions)),dim=1)

  #Get the index of the highest probability
  predicted_index,count=mode(np.array([a.argmax().item() for a in probabilities]))
  print(predicted_index)
  predicted_emotion=emotions[predicted_index[0]]

  print(f'predicted emotion: {predicted_emotion}')

In [ ]:
emotions('hello i amd very sad')

[1]
predicted emotion: sad


In [ ]:
emotions('nfenrbbeyncrgy')

[2]
predicted emotion: angry


In [ ]:
def summerize(text):
  summarization_pipeline=transformers.pipeline('summarization')

  #Summerize a text
  summary=summarization_pipeline(text,min_length=10,max_length=25)[0].get('summary_text')
  print(f'Summary:{summary}')

In [ ]:
summerize("The discovery of spectroscopy(opens in new tab), a discipline analyzing the ability of matter to split light into different wavelengths depending on its chemical composition, opened new and completely unexpected avenues of astronomical research in the second half of the 19th century. With spectroscopy, astronomers could study the chemical composition of celestial objects, first of those nearby, such as the moon and the sun, and later the more distant ones, including other stars and even galaxies. Suddenly, astronomy was not only about where things were located in the universe but also about what they were made of.")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Summary: The discovery of spectroscopy opened new and unexpected avenues of astronomical research in the second half of the 19th


In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=0fddd80ed5ae4125027d2eced80a9c6a2c5ad4ab0aab636905308e4b891aa0e4
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer,util
import spacy
from spacy.lang.en import STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import re
import os
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
def Similarity_Check(doc1,doc2):

  model=SentenceTransformer('roberta-base-nli-stsb-mean-tokens')
  query_embedding=model.encode(doc1)
  passage_embedding=model.encode(doc2)

  score="{:.4f}".format(util.cos_sim(query_embedding,passage_embedding).tolist()[0][0])
  score_norm=(float(score)+1)/(1+1)

  print(f"Similarity Score with  :","{:.4f}".format(score_norm))

  return score_norm

In [ ]:
Similarity_Check('do u still study ',"do u  study ")

Similarity Score with  : 0.8988


0.8988